## Imports

In [2]:
!pip install catboost
!pip install scipy==1.14.0
!pip install optuna

In [3]:
import pandas as pd
import numpy as np
import json
import scipy
from scipy.optimize import nnls
from scipy.optimize import lsq_linear

import ast

import torch
import matplotlib.pyplot as plt

import torch
from tqdm import tqdm
import optuna

from catboost import CatBoostRegressor, Pool
#from sklearn.metrics import mean_absolute_error, explained_variance_score, mean_absolute_percentage_error
#from sklearn.model_selection import train_test_split

In [4]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

## Data

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [6]:
vacancies = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/cost_of_skills/IT_vacancies_full.csv')

In [7]:
vacancies = vacancies.rename(columns={'Keys': 'Skills'})
vacancies['Skills'] = vacancies['Skills'].apply(lambda x: ast.literal_eval(x))
vacancies['Profarea names'] = vacancies['Profarea names'].apply(lambda x: eval(x))
vacancies['Professional roles'] = vacancies['Professional roles'].apply(lambda x: eval(x))

vacancies['Specializations'] = vacancies['Specializations'].apply(lambda x: eval(x))

In [8]:
# Applying the logic
vacancies['To'] = vacancies.apply(lambda row: row['From'] if pd.isna(row['To']) and not pd.isna(row['From']) and row['To'] != 0 else row['To'], axis=1)
vacancies['From'] = vacancies.apply(lambda row: row['To'] - 10000 if pd.isna(row['From']) and not pd.isna(row['To']) and row['To'] != 0 else row['From'], axis=1)

# Dropping rows where both From and To are NaN
vacancies.dropna(subset=['From', 'To'], how='all', inplace=True)

vacancies = vacancies[vacancies['To'] != 0]

vacancies = vacancies[vacancies['Salary'] == True]
vacancies = vacancies.drop('Salary', axis=1)

vacancies = vacancies.reset_index()

## Functions


In [9]:
################################################################################
# Обработка датафрейма и построение матрицы навыков
################################################################################

def get_all_skills(data: pd.DataFrame) -> list:
    """
    Собирает все уникальные навыки из столбца 'Skills' (список навыков в каждой строке).
    Возвращает список навыков (строки в нижнем регистре).
    """
    all_skills = []
    for row in data['Skills']:
        # row - это список навыков для строки датафрейма
        lowercase_list = [item.lower() for item in row]
        all_skills.extend(lowercase_list)

    return list(set(all_skills))  # set() чтобы убрать дубликаты, list() для явного возвращения списка


def make_cf(indices: list, skill_amount: int) -> list:
    """
    Создаёт вектор признаков для одной строки датафрейма.
    Напр. если у строки навыки: [0, 3, 5], а всего навыков skill_amount=10,
    то возвращаем вектор длины 10, где на позициях (0,3,5) будет 1, остальные 0.
    """
    cf = [0]*skill_amount
    for i in indices:
        cf[i] = 1
    return cf


def get_matrix(data: pd.DataFrame, skill_id_dict: dict) -> list:
    """
    Строит матрицу A размера [num_rows x skill_amount],
    где num_rows = число строк в data.
    """
    skill_amount = len(skill_id_dict)
    A = []
    for row in data['Skills']:
        # Преобразуем навыки строки в индексы
        indices = [skill_id_dict[skill.lower()] for skill in row]
        A.append(make_cf(indices, skill_amount))
    return A

## Experiments

### Данные о матрице


In [18]:
#собираем все Profarea names
prof_lst = []
for i in vacancies['Profarea names']:
    prof_lst += i
prof_dict = {i: [] for i in set(prof_lst)}

for i, j in enumerate(vacancies['Profarea names']):
    for k in j:
        prof_dict[k].append(i)

prof = list(prof_dict.keys())

In [25]:
# Вычисляем сингулярное разложение A
data =  vacancies.copy().dropna().reset_index()

data = data[['Skills', 'From', 'To']]
data = data.rename(columns={'From': 'from', 'To': 'to'})

data = data.dropna()
data = data[data['from'] != data['to']]
data = data[data['Skills'].apply(len) > 0]

# 1. Собираем полный список навыков:
all_skills = get_all_skills(data)

# 2. Создаём словарь маппинга навыка -> индекс:
skill_id_dict = {skill: idx for idx, skill in enumerate(all_skills)}

A = get_matrix(data, skill_id_dict)

A = torch.tensor(A, dtype=torch.float32, device=device)

U, S, Vh = torch.linalg.svd(A, full_matrices=False)

# Максимальное сингулярное число
max_singular_value = torch.max(S)

print("Сингулярные числа S:", S)
print("Максимальное сингулярное число:", max_singular_value)

Сингулярные числа S: tensor([5.1433e+01, 4.3381e+01, 3.6648e+01,  ..., 1.6528e-07, 1.0227e-07,
        4.6106e-08])
Максимальное сингулярное число: tensor(51.4330)


In [26]:
A.shape

torch.Size([8050, 5184])

In [24]:
for i in prof:
  data =  vacancies[vacancies['Profarea names'].apply(lambda x: any(i in profarea for profarea in x))].copy().dropna().reset_index()

  data = data[['Skills', 'From', 'To']]
  data = data.rename(columns={'From': 'from', 'To': 'to'})

  data = data.dropna()
  data = data[data['from'] != data['to']]
  data = data[data['Skills'].apply(len) > 0]


  # 1. Собираем полный список навыков:
  all_skills = get_all_skills(data)

  # 2. Создаём словарь маппинга навыка -> индекс:
  skill_id_dict = {skill: idx for idx, skill in enumerate(all_skills)}

  # 3. Собираем матрицу A:
  A = get_matrix(data, skill_id_dict)

  A = torch.tensor(A, dtype=torch.float32, device=device)

  U, S, Vh = torch.linalg.svd(A, full_matrices=False)

  # Максимальное сингулярное число
  max_singular_value = torch.max(S)

  print(f"Максимальное сингулярное число для {i} : {max_singular_value} и размер мартицы {A.shape}" )

Максимальное сингулярное число для Закупки : 5.958070278167725 и размер мартицы torch.Size([28, 154])
Максимальное сингулярное число для Туризм, гостиницы, рестораны : 4.561465740203857 и размер мартицы torch.Size([16, 84])
Максимальное сингулярное число для Высший менеджмент : 11.822216987609863 и размер мартицы torch.Size([99, 378])
Максимальное сингулярное число для Рабочий персонал : 13.550604820251465 и размер мартицы torch.Size([314, 560])
Максимальное сингулярное число для Начало карьеры, студенты : 16.421340942382812 и размер мартицы torch.Size([262, 676])
Максимальное сингулярное число для Инсталляция и сервис : 14.377412796020508 и размер мартицы torch.Size([397, 761])
Максимальное сингулярное число для Информационные технологии, интернет, телеком : 51.43290328979492 и размер мартицы torch.Size([8049, 5184])
Максимальное сингулярное число для Наука, образование : 7.347621440887451 и размер мартицы torch.Size([75, 340])
Максимальное сингулярное число для Административный персо

### NNLS

In [10]:
data =  vacancies.copy().dropna().reset_index()
data = data[['Skills', 'From', 'To']]
data = data.rename(columns={'From': 'from', 'To': 'to'})

data = data.dropna()
data = data[data['from'] != data['to']]
data = data[data['Skills'].apply(len) > 0]

In [11]:
 # 1. Собираем полный список навыков:
all_skills = get_all_skills(data)

# 2. Создаём словарь маппинга навыка -> индекс:
skill_id_dict = {skill: idx for idx, skill in enumerate(all_skills)}
# 3. Собираем матрицу A:
A = get_matrix(data, skill_id_dict)

b = data['to'].values

In [25]:
A.shape, b.shape[0], x.shape

(torch.Size([8050, 5184]), 8050, torch.Size([7038]))

In [16]:
A.shape[1]

5184

In [14]:
#Реализация с оптимизацией

A = torch.tensor(A, dtype=torch.float32, device=device)
b = torch.tensor(b, dtype=torch.float32, device=device)

# Инициализация переменных x с положительными значениями
x = torch.nn.Parameter(torch.full((A.shape[1],), 50000.0, device=device)) #x-количество навыков

# Параметры для штрафа
penalty_weight = 1  # Коэффициент штрафа (можно настроить)
threshold = 50000.0   # Пороговое значение для штрафа

# Оптимизатор
optimizer = torch.optim.Adam([x], lr=0.1)

# Планировщик обучения для уменьшения скорости обучения на плато
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
    optimizer, mode='min', factor=0.1, patience=1000, verbose=True # patience=1000 означает, что если потери не улучшаются в течение 1000 итераций, скорость обучения будет уменьшена.
)

# Цикл оптимизации
num_iterations = 10000
for iteration in range(num_iterations):
    optimizer.zero_grad()

    # Вычисление невязки системы уравнений
    residual = (torch.matmul(A, x) - b).abs()
    loss_equations = torch.mean(residual)

    # Штраф для x_i < 10,000
    penalty = penalty_weight * torch.mean((torch.relu(residual - threshold)))

    # Общая функция потерь
    loss = loss_equations + penalty

    # Обратное распространение ошибки
    loss.backward()
    optimizer.step()

    # Принудительное ограничение x_i >= 0
    with torch.no_grad():
        x.clamp_(min=0)

    # Обновление планировщика
    scheduler.step(loss)

    # Периодический вывод информации
    if (iteration + 1) % 500 == 0:
        print(f"Итерация {iteration + 1}, Потери: {loss.item():.2f}, Penalty: {penalty}, Equations: {loss_equations}")

# Вывод решения
print("\nРешение с штрафом для x_i < 10,000:")
for i, xi in enumerate(x.detach().cpu().numpy(), start=1):
    print(f"x{i} = {xi:.2f}")

<ipython-input-14-67f04dc6fff7>:3: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  A = torch.tensor(A, dtype=torch.float32, device=device)
<ipython-input-14-67f04dc6fff7>:4: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  b = torch.tensor(b, dtype=torch.float32, device=device)
/usr/local/lib/python3.11/dist-packages/torch/optim/lr_scheduler.py:62: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(


Выходные данные были обрезаны до нескольких последних строк (5000).
x185 = 48984.38
x186 = 48984.38
x187 = 48984.38
x188 = 48984.38
x189 = 48984.38
x190 = 48984.38
x191 = 48984.74
x192 = 48984.38
x193 = 48994.65
x194 = 48984.38
x195 = 48984.38
x196 = 48984.38
x197 = 48984.38
x198 = 48984.38
x199 = 48984.38
x200 = 48984.38
x201 = 48984.38
x202 = 48984.38
x203 = 48984.38
x204 = 48984.38
x205 = 48984.38
x206 = 48984.38
x207 = 51015.62
x208 = 48984.38
x209 = 48984.38
x210 = 48984.38
x211 = 48984.38
x212 = 48984.38
x213 = 48984.38
x214 = 48984.38
x215 = 48984.38
x216 = 48984.38
x217 = 48984.38
x218 = 48984.38
x219 = 48984.38
x220 = 48984.38
x221 = 48984.38
x222 = 48984.38
x223 = 48984.38
x224 = 48984.38
x225 = 48984.38
x226 = 48984.38
x227 = 48984.38
x228 = 48984.38
x229 = 48984.38
x230 = 48984.38
x231 = 48984.38
x232 = 48984.38
x233 = 49999.41
x234 = 48984.38
x235 = 48984.38
x236 = 48985.77
x237 = 48984.38
x238 = 48984.38
x239 = 48984.38
x240 = 48984.38
x241 = 48984.38
x242 = 48984.38
x243

In [15]:
np.abs(A.cpu().numpy()@(x.detach().cpu().numpy())-b.cpu().numpy()).mean()

210468.06

In [21]:
import numpy as np
from scipy.optimize import lsq_linear

def train_nnls(A, b):
    """
    Обучает модель NNLS (Non-Negative Least Squares), возвращает найденные веса (x),
    а также метрики MSE (loss) и MAE на исходных данных.

    Параметры:
    -----------
    A : array-like, shape (n_samples, n_features)
        Матрица признаков (исходные данные).
    b : array-like, shape (n_samples,)
        Целевой вектор.

    Возвращает:
    -----------
    x : ndarray, shape (n_features,)
        Найденные веса в исходном масштабе.
    loss : float
        MSE (Mean Squared Error) на исходных данных.
    mae : float
        MAE (Mean Absolute Error) на исходных данных.
    """

    A = torch.tensor(A, dtype=torch.float32, device=device)
    b = torch.tensor(b, dtype=torch.float32, device=device)

    # Инициализация переменных x с положительными значениями
    x = torch.nn.Parameter(torch.full((A.shape[1],), 50000.0, device=device)) #x-количество навыков

    # Параметры для штрафа
    penalty_weight = 1  # Коэффициент штрафа (можно настроить)
    threshold = 50000.0   # Пороговое значение для штрафа

    # Оптимизатор
    optimizer = torch.optim.Adam([x], lr=0.1)

    # Планировщик обучения для уменьшения скорости обучения на плато
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
        optimizer, mode='min', factor=0.1, patience=1000, verbose=True # patience=1000 означает, что если потери не улучшаются в течение 1000 итераций, скорость обучения будет уменьшена.
    )

    # Цикл оптимизации
    num_iterations = 10000
    for iteration in range(num_iterations):
        optimizer.zero_grad()

        # Вычисление невязки системы уравнений
        residual = (torch.matmul(A, x) - b).abs()
        loss_equations = torch.mean(residual)

        # Штраф для x_i < 10,000
        penalty = penalty_weight * torch.mean((torch.relu(residual - threshold)))

        # Общая функция потерь
        loss = loss_equations + penalty

        # Обратное распространение ошибки
        loss.backward()
        optimizer.step()

        # Принудительное ограничение x_i >= 0
        with torch.no_grad():
            x.clamp_(min=0)

        # Обновление планировщика
        scheduler.step(loss)

    final_loss = np.abs(A.cpu().numpy()@(x.detach().cpu().numpy())-b.cpu().numpy()).mean()

    return final_loss


In [22]:
dic_of_mae = {}
for i in prof:
  data =  vacancies[vacancies['Profarea names'].apply(lambda x: any(i in profarea for profarea in x))].copy().dropna().reset_index()

  data = data[['Skills', 'From', 'To']]
  data = data.rename(columns={'From': 'from', 'To': 'to'})

  data = data.dropna()
  data = data[data['from'] != data['to']]
  data = data[data['Skills'].apply(len) > 0]


  # 1. Собираем полный список навыков:
  all_skills = get_all_skills(data)

  # 2. Создаём словарь маппинга навыка -> индекс:
  skill_id_dict = {skill: idx for idx, skill in enumerate(all_skills)}

  # 3. Собираем матрицу A:
  A = get_matrix(data, skill_id_dict)
  final_loss = train_nnls(A, data['to'].values)
  #mae_value = calculate_mae_for_profarea(vacancies, i)
  dic_of_mae[i] = final_loss
  print(f"{i} -> {final_loss}")

/usr/local/lib/python3.11/dist-packages/torch/optim/lr_scheduler.py:62: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(


Закупки -> 345558.34375


/usr/local/lib/python3.11/dist-packages/torch/optim/lr_scheduler.py:62: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(


Туризм, гостиницы, рестораны -> 224572.5625


/usr/local/lib/python3.11/dist-packages/torch/optim/lr_scheduler.py:62: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(


...

Государственная служба, некоммерческие организации -> 296913.40625


/usr/local/lib/python3.11/dist-packages/torch/optim/lr_scheduler.py:62: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(


Юристы -> 163015.625


/usr/local/lib/python3.11/dist-packages/torch/optim/lr_scheduler.py:62: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(


Бухгалтерия, управленческий учет, финансы предприятия -> 237958.75


### Pytorch

In [ ]:
import torch
import numpy as np

def train_nnls_with_penalty(
    A,
    b,
    penalty_weight=1.0,
    threshold=50000.0,
    lr=0.1,
    num_iterations=10000,
    patience=1000,
    initial_x_value=50000.0,
    device='cpu'
):
    """
    Приближённое решение задачи NNLS (Non-negative Least Squares) в PyTorch
    с добавлением штрафа для больших значений x и ограничением x_i >= 0.

    Параметры
    ---------
    A : array-like, shape (n_samples, n_features)
        Матрица системы (признаки).
    b : array-like, shape (n_samples,)
        Вектор правой части уравнений (целевая переменная).
    penalty_weight : float
        Коэффициент штрафа (масштабирует вклад штрафа в функцию потерь).
    threshold : float
        Пороговое значение, выше которого срабатывает штраф.
    lr : float
        Начальный шаг обучения (learning rate) для Adam.
    num_iterations : int
        Количество итераций оптимизации.
    patience : int
        Количество итераций без улучшения loss, после которых
        ReduceLROnPlateau уменьшает lr.
    initial_x_value : float
        Начальное значение, с которого инициализируется вектор x.
    device : str
        'cpu' или 'cuda' - устройство, на котором будут выполняться вычисления.

    Возвращает
    ----------
    x_final : ndarray, shape (n_features,)
        Найденное решение (количество навыков), принудительно неотрицательное.
    mse : float
        Среднеквадратичная ошибка (MSE) на исходных данных.
    mae : float
        Средняя абсолютная ошибка (MAE) на исходных данных.
    """
    # Переводим исходные данные в тензоры на нужном устройстве
    A_torch = torch.tensor(A, dtype=torch.float32, device=device)
    b_torch = torch.tensor(b, dtype=torch.float32, device=device)

    # Инициализация x (можно указать конкретный размер, если известно число столбцов)
    n_features = A_torch.shape[1] if A_torch.dim() > 1 else 1
    x_param = torch.nn.Parameter(
        torch.full((n_features,), initial_x_value, device=device)
    )

    # Оптимизатор
    optimizer = torch.optim.Adam([x_param], lr=lr)

    # Планировщик обучения (уменьшение lr на плато)
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
        optimizer,
        mode='min',
        factor=0.1,
        patience=patience,
        verbose=True
    )

    # Цикл оптимизации
    for iteration in range(num_iterations):
        optimizer.zero_grad()

        # Вычисление невязки системы уравнений (средняя абсолютная)
        residual = (A_torch @ x_param - b_torch).abs()
        loss_equations = torch.mean(residual)

        # Штраф для «больших» значений (если residual превышает threshold)
        penalty = penalty_weight * torch.mean(torch.relu(residual - threshold))

        # Общая функция потерь
        loss = loss_equations + penalty

        # Обратное распространение
        loss.backward()
        optimizer.step()

        # Принудительное ограничение x_i >= 0
        with torch.no_grad():
            x_param.clamp_(min=0)

        # Обновление планировщика
        scheduler.step(loss)

        # Периодический вывод
        if (iteration + 1) % 500 == 0:
            print(f"Итерация {iteration+1}, Loss: {loss.item():.2f}, "
                  f"Penalty: {penalty.item():.2f}, Equations: {loss_equations.item():.2f}")

    # Приводим итоговый x к numpy
    x_final = x_param.detach().cpu().numpy()

    # Считаем итоговые предсказания и метрики (MSE, MAE)
    with torch.no_grad():
        y_pred = A_torch @ x_param

    y_pred_np = y_pred.detach().cpu().numpy()
    b_np = b_torch.detach().cpu().numpy()

    mse = np.mean((y_pred_np - b_np) ** 2)
    mae = np.mean(np.abs(y_pred_np - b_np))

    return x_final, mse, mae
